In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [15]:
def create_land_prices_df():
    basedir = './공시지가/'
    filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
    
    df = pd.read_csv(basedir + filenames[0], encoding='euc-kr', usecols=[2,4,5,6,8,10])
    
    df = df[df['기준년도'] >= 2006]
    
    return df

In [16]:
land_prices_df = create_land_prices_df()
print(land_prices_df.shape)
land_prices_df.head()

(13772806, 6)


,법정동명,특수지구분명,지번,기준년도,공시지가,표준지여부
16,서울특별시 종로구 청운동,일반,1,2006,2420000,N
17,서울특별시 종로구 청운동,일반,1,2007,2580000,N
18,서울특별시 종로구 청운동,일반,1,2008,2830000,N
19,서울특별시 종로구 청운동,일반,1,2009,2770000,N
20,서울특별시 종로구 청운동,일반,1,2010,2840000,N


In [17]:
def landnum_modifier(x):
    # x is a string
    
    splitted = x.split('-')
    if len(splitted) == 1:
        return x+'-0'
    elif len(splitted) == 2:
        return x
    else:
        raise ValueError('myerror')

In [18]:
land_prices_df['지번주소'] = land_prices_df['법정동명'] + ' ' + land_prices_df['지번'].apply(landnum_modifier)
land_prices_df.head()

,법정동명,특수지구분명,지번,기준년도,공시지가,표준지여부,지번주소
16,서울특별시 종로구 청운동,일반,1,2006,2420000,N,서울특별시 종로구 청운동 1-0
17,서울특별시 종로구 청운동,일반,1,2007,2580000,N,서울특별시 종로구 청운동 1-0
18,서울특별시 종로구 청운동,일반,1,2008,2830000,N,서울특별시 종로구 청운동 1-0
19,서울특별시 종로구 청운동,일반,1,2009,2770000,N,서울특별시 종로구 청운동 1-0
20,서울특별시 종로구 청운동,일반,1,2010,2840000,N,서울특별시 종로구 청운동 1-0


In [19]:
land_prices_df.drop(columns=['법정동명', '지번'], inplace=True)

In [20]:
land_prices_df.head()

,특수지구분명,기준년도,공시지가,표준지여부,지번주소
16,일반,2006,2420000,N,서울특별시 종로구 청운동 1-0
17,일반,2007,2580000,N,서울특별시 종로구 청운동 1-0
18,일반,2008,2830000,N,서울특별시 종로구 청운동 1-0
19,일반,2009,2770000,N,서울특별시 종로구 청운동 1-0
20,일반,2010,2840000,N,서울특별시 종로구 청운동 1-0


In [21]:
land_prices_df.rename(columns={'기준년도':'년'}, inplace=True)

In [22]:
print(land_prices_df.shape)
land_prices_df.drop_duplicates(subset=['지번주소', '년'], keep='first', inplace=True)
land_prices_df.shape

(13772806, 5)


(13701054, 5)

In [23]:
land_prices_df.head()

,특수지구분명,년,공시지가,표준지여부,지번주소
16,일반,2006,2420000,N,서울특별시 종로구 청운동 1-0
17,일반,2007,2580000,N,서울특별시 종로구 청운동 1-0
18,일반,2008,2830000,N,서울특별시 종로구 청운동 1-0
19,일반,2009,2770000,N,서울특별시 종로구 청운동 1-0
20,일반,2010,2840000,N,서울특별시 종로구 청운동 1-0


In [31]:
count = 0
for addr in tqdm(land_prices_df['지번주소'].unique()):
    addr_df = land_prices_df[land_prices_df['지번주소'] == addr]
    
    if addr_df.shape[0] != (2020-2006 + 1):
        count += 1
        '''print(addr_df.shape)
        display(addr_df)
        
        
        raise ValueError('myerror')'''

NameError: name 'land_prices_df' is not defined

In [28]:
main_data_df = pd.read_csv('main_data_ver2.2.csv')
print(main_data_df.shape)
main_data_df.head()

(4559866, 27)


,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,지번주소,대장구분코드,대장구분명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반


In [29]:
selected_df = land_prices_df[land_prices_df['지번주소'].isin(main_data_df['지번주소'].unique())]
print(selected_df.shape)
selected_df.head()

(1594633, 5)


,특수지구분명,년,공시지가,표준지여부,지번주소
16,일반,2006,2420000,N,서울특별시 종로구 청운동 1-0
17,일반,2007,2580000,N,서울특별시 종로구 청운동 1-0
18,일반,2008,2830000,N,서울특별시 종로구 청운동 1-0
19,일반,2009,2770000,N,서울특별시 종로구 청운동 1-0
20,일반,2010,2840000,N,서울특별시 종로구 청운동 1-0


In [33]:
selected_df.to_csv('land_prices_ver_1.csv', index=False)